# ETL (Extraction, Transform and Load) 

In [21]:
# Importar librerías

import pandas as pd
from summarytools import dfSummary
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 1. Extraction

In [2]:
# Cargar archivos que se van a utilizar

df_hechos=pd.read_excel('./dataset/homicidios.xlsx', sheet_name='HECHOS', na_values='SD')
df_victimas=pd.read_excel('./dataset/homicidios.xlsx', sheet_name='VICTIMAS', na_values='SD')

Al cargar los DataFrames con el parámetro 'na_values' para las palabras 'SD' se muestren como valores nulos, intentaremos hacer la prueba si funcionó correctamente. En este caso, si visualizamos el DataSet, El que tiene **"ID=2016-0151"** tenía una dimensión **"LUGAR_DE_HECHO == SD"**. Por consiguiente, se valida de que fue transformado correctamente a NaN.

In [3]:
df_hechos[df_hechos['ID'] == '2016-0151']

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20.0,NaN,CALLE,...,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,NaN


### 2. Transformation

**Tabla de "HECHOS"**

In [4]:
#Visualizamos las prim
df_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,NaN
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [5]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    float64       
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

Dentro de la exploración y sobre lo que nos están solicitado, vamos a eliminar las dimensiones (columnas) que no se utilizarán dentro del análisis.

In [6]:
#Eliminamos columnas irrelevantes para el análisis
df_hechos.drop(columns=['AAAA','MM','DD','HORA','LUGAR_DEL_HECHO','Calle','Altura','Cruce','Dirección Normalizada','PARTICIPANTES'], inplace=True)

In [7]:
#Visualizamos la cantidad de nulos que tenemos en este dataset
df_hechos.isnull().sum()

ID                0
N_VICTIMAS        0
FECHA             0
HH                1
TIPO_DE_CALLE     0
COMUNA            0
XY (CABA)         0
pos x             0
pos y             0
VICTIMA           9
ACUSADO          23
dtype: int64

Dentro de la revisión de las columnas, cambiamos a valores "NaN", los valores descritos posteriormente

In [8]:
#Reemplazamos los valores 0 de la 'COMUNA' a nulos 
df_hechos.loc[df_hechos['COMUNA'] == 0, 'COMUNA'] = np.nan

#Reemplazamos a valores nulos, la longitud y latitud
df_hechos.loc[df_hechos['XY (CABA)'] == 'Point (. .)', 'XY (CABA)'] = np.nan
df_hechos.loc[df_hechos['pos x'] == '.', 'pos x'] = np.nan
df_hechos.loc[df_hechos['pos y'] == '.', 'pos y'] = np.nan

In [9]:
df_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             696 non-null    object        
 1   N_VICTIMAS     696 non-null    int64         
 2   FECHA          696 non-null    datetime64[ns]
 3   HH             695 non-null    float64       
 4   TIPO_DE_CALLE  696 non-null    object        
 5   COMUNA         694 non-null    float64       
 6   XY (CABA)      682 non-null    object        
 7   pos x          684 non-null    object        
 8   pos y          684 non-null    object        
 9   VICTIMA        687 non-null    object        
 10  ACUSADO        673 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 59.9+ KB


In [10]:
df_hechos.head()

,ID,N_VICTIMAS,FECHA,HH,TIPO_DE_CALLE,COMUNA,XY (CABA),pos x,pos y,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,4.0,AVENIDA,8.0,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO,AUTO
1,2016-0002,1,2016-01-02,1.0,GRAL PAZ,9.0,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,7.0,AVENIDA,1.0,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO,AUTO
3,2016-0004,1,2016-01-10,0.0,AVENIDA,8.0,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO,NaN
4,2016-0005,1,2016-01-21,5.0,AVENIDA,1.0,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO,PASAJEROS


Cambiamos el tipo de algunas dimensiones (columnas)

In [11]:
#Cambiamos el tipo de algunas dimensiones(columnas) a Categóricos
df_hechos['HH'] = df_hechos['HH'].astype('category')
df_hechos['TIPO_DE_CALLE'] = df_hechos['TIPO_DE_CALLE'].astype('category')
df_hechos['COMUNA'] = df_hechos['COMUNA'].astype('category')
df_hechos['VICTIMA'] = df_hechos['VICTIMA'].astype('category')
df_hechos['ACUSADO'] = df_hechos['ACUSADO'].astype('category')

In [12]:
#Cambiamos el tipo de algunas dimensiones(columnas) a Float
df_hechos['pos x'] = df_hechos['pos x'].astype('float')
df_hechos['pos y'] = df_hechos['pos y'].astype('float')


In [13]:
#Cambiamos el tipo de la fecha
df_hechos["FECHA"]=pd.to_datetime(df_hechos["FECHA"], format='%y/%m/%d', errors='coerce').dt.date

Al notar que los nombres de las columnas no estaban bien emparejados puesto que, algunos nombres estaban en minúscula, por eso que finalmente nos decidimos en ponerlo todo en mayúscula.

P.D: En esta parte incluímos también las columnas del otro dataframe para mayor comodidad.

In [14]:
# Renombramos las columnas
new_names = {'ID': 'ID_HECHO','pos x':'LONGITUD','pos y':'LATITUD',
                  'HH':'HORA','VICTIMA':'VEHICULO_VICTIMA','ACUSADO':'VEHICULO_ACUSADO',
                  'Rango_etario ':'RANGO_ETARIO'}
                 
# Reemplazamos nombres de columnas
df_hechos = df_hechos.rename(columns = new_names)

In [15]:
df_hechos.columns

Index(['ID_HECHO', 'N_VICTIMAS', 'FECHA', 'HORA', 'TIPO_DE_CALLE', 'COMUNA',
       'XY (CABA)', 'LONGITUD', 'LATITUD', 'VEHICULO_VICTIMA',
       'VEHICULO_ACUSADO'],
      dtype='object')

Analizamos las dimensiones más a detalle.

 **Dimensión 'TIPO_DE_CALLE'**

In [16]:
#Visualizamos las categorías de TIPO_DE_CALLE
df_hechos['TIPO_DE_CALLE'].value_counts()

TIPO_DE_CALLE
AVENIDA      429
CALLE        136
AUTOPISTA     66
GRAL PAZ      65
Name: count, dtype: int64

En este caso vemos que existe la categoría 'GRAL PAZ', investigando se refiere a la Av. General Paz, por ende, a todos los valores con este nombre, se le reemplazará a 'AVENIDA'.

In [17]:
#Reemplazamos 'GRAL PAZ' por 'AVENIDA' 
df_hechos['TIPO_DE_CALLE'].replace('GRAL PAZ', 'AVENIDA', inplace=True)

C:\Users\Rafael Farfan\AppData\Local\Temp\ipykernel_37220\2495949059.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_hechos['TIPO_DE_CALLE'].replace('GRAL PAZ', 'AVENIDA', inplace=True)
C:\Users\Rafael Farfan\AppData\Local\Temp\ipykernel_37220\2495949059.py:2: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_

Por último realizamos un resumen descriptivo de los datos.

In [24]:
dfSummary(df_hechos)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,ID_HECHO[object],1. 2016-00012. 2019-00503. 2019-00424. 2019-00435. 2019-00446. 2019-00457. 2019-00468. 2019-00479. 2019-004810. 2019-004911. other,1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)1 (0.1%)686 (98.6%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/H5lhTAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADXElEQVR4nO3cwWkbYRRG0TdGOPZGQhhcQtKDi0ixKUJFpAdjPIvY8WaySBYpINwB5ZwK/sWFh2A+Ldu2DRRu9n4A/w+xkREbGbGRERsZsZERGxmxkREbGbGRuVmW5cuyLHd7P4TrdzMzn2fm094P4fo5o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRG5mZmvs/Mz70fwvVb/IsRFWeUjNjIWFeRsa4i44ySERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZExeCFj8ELGGSUjNjIGL2QMXsg4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjNjIWFeRsa4i44ySERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZGxriJjXUXGGSUjNjIGL2QMXsg4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZGxriJjXUXGGSUjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyYiMjNjLWVWSsq8g4o2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkXFGyYiNjMELGYMXMs4oGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZgxcyBi9knFEyy8zcz8zt3g/hqn1s2/Z2eHh4+Ho8Hs97v4brta7ry7Is3w7H4/H89PT0djqd3vd+FNfn9fX17nK5nJ+fn28PMzOn0+n98fHxx94P42rdz/iBQEhsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERuYw8/szkL0fwnX6u63Duq4vl8vlPH8+A4F/bV3Xl5n58Fk4hY9t296sq8j4gUBGbGTERkZsZMRGRmxkxEZGbGTERuYXwgdpOf758ysAAAAASUVORK5CYII="">",0(0.0%)
2,N_VICTIMAS[int64],Mean (sd) : 1.0 (0.2)min < med < max:1.0 < 1.0 < 3.0IQR (CV) : 0.0 (5.7),3 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/H5lhTAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABs0lEQVR4nO3XQWrCUBCH8ZkiqQ/khZCaY3gAD+FhPYRbTxLkZdGoVF53hVJaTbToX77fNplhFh8RPedswKN7ufcBwCUm515w92Bmxcj9x5xzP3IW+PJnqO4e6rpexRirMcu7rtu5+5pYca1zX9Qixlgtl8u+LMv9kMUppelms6nati3MjFBxlbM//WZmZVnum6Z5H7E/jJgBfuDPFCQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiQQKiRM7n3AM3L3YGbFyPFjzrm/5T3PgFBvzN1DXderGGM1Zr7rup27r4n1u4tCTSlNhy5OKU1Pp9PEzKK7Dz5MWJzNZvPFYvERQjgMGez7/nW73c7btn1z9+6/DnxQh5zz/reHnwblSFGwSZ17AAAAAElFTkSuQmCC"">",0(0.0%)
3,FECHA[object],1. 2018-12-182. 2016-11-263. 2018-12-224. 2018-02-175. 2016-02-286. 2016-02-177. 2018-04-278. 2016-06-139. 2017-11-1910. 2018-12-1211. other,3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)3 (0.4%)666 (95.7%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/H5lhTAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADfUlEQVR4nO3cwWoTUQCG0ZsSit1MCIE8TB6iD+tD5G1CyCxs6SYujCtNFaxfMJ6zEtzM4mMyXu7v4nw+Dyg83PoB+H+IjYzYyIiNjNjIiI2M2MiIjYzYyCzGGE9jjHE+n19u/CzcueVms3keY4zFYvFZcPxND9M0radpWo8xHm/9MNw332xkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERmY5z/Px8ue3mz4Jd88GgczCf5lFxZuNjHUVGesqMo4+yIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIGLyQcS2cjMELGW82MgYvZAxeyDj6ICM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjY/BCxrVwMgYvZHyzkfEzSsa6iox1FRnfbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryLgWTsa6iow3GxmDFzIGL2QcfZARGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkTF4IeNaOBmDFzLebGQMXsgYvJBx9EFGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryNggkLGuIuPNRsa6iox1FRlHH2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyLgWTsbghYw3GxmDFzIGL2QcfZARGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkTF4IeNaOBmDFzK+2cj4GSVjXUXGuoqMbzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjLWVWRcCydjXUXGm42MwQsZgxcyjj7IiI2M2MiIjYzYyIi

**Tabla de "VICTIMAS"**

In [25]:
#Visualizamos las prim
df_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,NaN
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29.0,2016-02-01 00:00:00


In [26]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  706 non-null    object        
 6   VICTIMA              708 non-null    object        
 7   SEXO                 711 non-null    object        
 8   EDAD                 664 non-null    float64       
 9   FECHA_FALLECIMIENTO  649 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 56.1+ KB


Así como en la primera tabla, dentro de la exploración y sobre lo que nos están solicitado, vamos a eliminar las dimensiones (columnas) que no se utilizarán dentro del análisis.

In [28]:

#Eliminamos columnas irrelevantes para el análisis
df_victimas.drop(columns=['AAAA','MM','DD','FECHA_FALLECIMIENTO'], inplace=True)

In [30]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   ID_hecho  717 non-null    object        
 1   FECHA     717 non-null    datetime64[ns]
 2   ROL       706 non-null    object        
 3   VICTIMA   708 non-null    object        
 4   SEXO      711 non-null    object        
 5   EDAD      664 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 33.7+ KB


Analizamos que la edad junto a otras dimensiones serán categóricas, por ende, se crea una función para asignar el rango etario.

In [38]:
#Creamos la función de asignar rango etario
def asignar_rango_etario(edad):
    if edad < 18:
        return "1) 0 a 17 años"
    elif 18 <= edad <= 29:
        return "2) 18 a 29 años"
    elif 30 <= edad <= 39:
        return "3) 30 a 39 años"
    elif 40 <= edad <= 49:
        return "4) 45 a 49 años"
    elif 50 <= edad <= 59:
        return "5) 50 a 59 años"
    elif edad >= 60:
        return "6) 60 años o mas"
    else:
        return np.nan

In [39]:
#Aplicamos la función de rango etario y lo agregamos a una dimensión
df_victimas['RANGO_ETARIO'] = df_victimas['EDAD'].apply(lambda x: asignar_rango_etario(x))
df_victimas.head()

,ID_hecho,FECHA,ROL,VICTIMA,SEXO,EDAD,RANGO_ETARIO
0,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19.0,2) 18 a 29 años
1,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70.0,6) 60 años o mas
2,2016-0003,2016-01-03,CONDUCTOR,MOTO,MASCULINO,30.0,3) 30 a 39 años
3,2016-0004,2016-01-10,CONDUCTOR,MOTO,MASCULINO,18.0,2) 18 a 29 años
4,2016-0005,2016-01-21,CONDUCTOR,MOTO,MASCULINO,29.0,2) 18 a 29 años


In [40]:
# Renombramos las columnas
new_names2 = {'ID_hecho':'ID_HECHO','ROL':'ROL_VICTIMA','VICTIMA':'VEHICULO_VICTIMA','SEXO':'SEXO_VICTIMA','EDAD':'EDAD_VICTIMA'}

# Reemplazamos nombres de columnas
df_victimas = df_victimas.rename(columns=new_names2)

In [42]:
df_victimas.columns

Index(['ID_HECHO', 'FECHA', 'ROL_VICTIMA', 'VEHICULO_VICTIMA', 'SEXO_VICTIMA',
       'EDAD_VICTIMA', 'RANGO_ETARIO'],
      dtype='object')

Cambiamos el tipo de algunas dimensiones (columnas)

In [44]:
#Cambiamos el tipo de algunas dimensiones(columnas) a Categóricos
df_victimas['ROL_VICTIMA'] = df_victimas['ROL_VICTIMA'].astype('category')
df_victimas['VEHICULO_VICTIMA'] = df_victimas['VEHICULO_VICTIMA'].astype('category')
df_victimas['SEXO_VICTIMA'] = df_victimas['SEXO_VICTIMA'].astype('category')
df_victimas['RANGO_ETARIO'] = df_victimas['RANGO_ETARIO'].astype('category')

In [46]:
#Cambiamos el tipo de la fecha
df_victimas["FECHA"]=pd.to_datetime(df_victimas["FECHA"], format='%y/%m/%d', errors='coerce').dt.date

In [45]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID_HECHO          717 non-null    object        
 1   FECHA             717 non-null    datetime64[ns]
 2   ROL_VICTIMA       706 non-null    category      
 3   VEHICULO_VICTIMA  708 non-null    category      
 4   SEXO_VICTIMA      711 non-null    category      
 5   EDAD_VICTIMA      664 non-null    float64       
 6   RANGO_ETARIO      664 non-null    category      
dtypes: category(4), datetime64[ns](1), float64(1), object(1)
memory usage: 20.6+ KB


### 3. Load

In [53]:
df_hechos.to_excel('dataset/hechos.xlsx', index=False)

In [52]:
df_victimas.to_excel('dataset/victimas.xlsx', index=False)